# Serving TensorFlow models

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
print(pd.__version__)

0.23.3


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.9.0


In [5]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [6]:
# a small sane check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


## Loading and validating our model

In [9]:
!curl -O https://raw.githubusercontent.com/DJCordhose/ai/master/notebooks/manning/model/insurance.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  175k  100  175k    0     0   449k      0 --:--:-- --:--:-- --:--:--  449k


In [10]:
model = tf.keras.models.load_model('insurance.hdf5')

### Descison Boundaries for 2 Dimensions

![Descison Boundaries for 2 Dimensions](https://djcordhose.github.io/ai/img/manning/nn-reg.png)

In [60]:
# a little sane check, does it work at all?

# within this code, we expect Olli to be a green customer with a high prabability
# 0: red
# 1: green
# 2: yellow

olli_data = [100, 47, 10]

X = np.array([olli_data])
model.predict(X)

array([[0.0027608 , 0.8720881 , 0.12515119]], dtype=float32)

## Converting our Keras Model to an Estimator Model which is what Keras wants for serving

In [61]:
# https://cloud.google.com/blog/products/gcp/new-in-tensorflow-14-converting-a-keras-model-to-a-tensorflow-estimator
estimator_model = tf.keras.estimator.model_to_estimator(keras_model=model)

In [62]:
# it still works the same, with a different style of API, though
x = {"hidden1_input": X}
list(estimator_model.predict(input_fn=tf.estimator.inputs.numpy_input_fn(x, shuffle=False)))

[{'softmax': array([0.0027608 , 0.8720881 , 0.12515119], dtype=float32)}]

## Preparing our model for serving

In [63]:
# https://www.tensorflow.org/serving/
# https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/setup.md#tensorflow-serving-python-api-pip-package-pip

!pip install tensorflow-serving-api

thinc 6.11.2 requires hypothesis<3,>=2, which is not installed.
thinc 6.11.2 requires msgpack-python==0.5.4, which is not installed.
thinc 6.11.2 has requirement msgpack-numpy==0.4.1, but you'll have msgpack-numpy 0.4.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
spacy 2.0.11 has requirement regex==2017.4.5, but you'll have regex 2018.6.21 which is incompatible.
spacy 2.0.11 has requirement thinc<6.11.0,>=6.10.1, but you'll have thinc 6.11.2 which is incompatible.
Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\tensorflow\\__init__.py'
Consider using the `--user` option or check the permissions.

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading